In [1]:
import sys; sys.path.append('..'); sys.path.append('../..')
from common.postgresql import PostgresConnector
db = PostgresConnector()
read_sql = db.read_sql
sql = db.execute_sql

In [2]:
sql("""CREATE OR REPLACE FUNCTION cvm.drop_view_any(schema_name TEXT, view_name TEXT)
RETURNS TEXT AS $$
DECLARE
    v_type CHAR;
BEGIN
    -- Correção: relnamespace está em pg_class (c) e liga-se ao oid de pg_namespace (n)
    SELECT c.relkind INTO v_type
    FROM pg_class c
    JOIN pg_namespace n ON c.relnamespace = n.oid
    WHERE n.nspname = schema_name 
      AND c.relname = view_name;

    IF v_type = 'v' THEN
        EXECUTE format('DROP VIEW %I.%I CASCADE', schema_name, view_name);
        RETURN 'VIEW ' || schema_name || '.' || view_name || ' removida.';
    ELSIF v_type = 'm' THEN
        EXECUTE format('DROP MATERIALIZED VIEW %I.%I CASCADE', schema_name, view_name);
        RETURN 'MATERIALIZED VIEW ' || schema_name || '.' || view_name || ' removida.';
    ELSE
        RETURN 'Objeto não encontrado ou não é uma View.';
    END IF;
END;
$$ LANGUAGE plpgsql;""")

Operação realizada com sucesso.


In [2]:
tabelas_cvm = read_sql("SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA='cvm'")
for _, x in tabelas_cvm.iterrows():
    table_name = x.iloc[0]
    # if 'fi_cad' in table_name:
    if 'cda_fi' in table_name:
        df = db.read_sql("SELECT * FROM cvm.{} LIMIT 10".format(table_name))
        print(table_name, 'Colunas: ', df.columns.tolist())
        # df = db.read_sql("SELECT * FROM cvm.{} WHERE cnpj_fundo = '42.195.812/0001-80'".format(table_name))
        # print(table_name, 'Linhas: ', len(df))
        # df = db.read_sql("SELECT * FROM cvm.{} WHERE cnpj_fundo = '42.195.812/0001-80'".format(table_name))
        # print(table_name, 'Linhas: ', len(df))

fi_doc_cda_fie Colunas:  ['tp_fundo_classe', 'cnpj_fundo_classe', 'denom_social', 'dt_comptc', 'id_doc', 'vl_patrim_liq', 'tp_aplic', 'tp_ativo', 'emissor_ligado', 'tp_negoc', 'qt_venda_negoc', 'vl_venda_negoc', 'qt_aquis_negoc', 'vl_aquis_negoc', 'qt_pos_final', 'vl_merc_pos_final', 'vl_custo_pos_final', 'dt_confid_aplic', 'cd_ativo', 'ds_ativo', 'dt_venc', 'pf_pj_emissor', 'cpf_cnpj_emissor', 'emissor', 'risco_emissor', 'cd_selic', 'dt_ini_vigencia', 'cd_pais', 'pais', 'cd_bv_merc', 'bv_merc', '__file']
fi_doc_cda_fie_confid Colunas:  ['tp_fundo_classe', 'cnpj_fundo_classe', 'denom_social', 'dt_comptc', 'id_doc', 'tp_aplic', 'vl_venda_negoc', 'vl_aquis_negoc', 'vl_merc_pos_final', 'vl_custo_pos_final', 'dt_confid_aplic', '__file']
fi_doc_cda_fiim Colunas:  ['tp_fundo', 'cnpj_fundo', 'denom_social', 'dt_comptc', 'id_doc', 'vl_patrim_liq', 'tp_aplic', 'tp_ativo', 'emissor_ligado', 'tp_negoc', 'qt_venda_negoc', 'vl_venda_negoc', 'qt_aquis_negoc', 'vl_aquis_negoc', 'qt_pos_final', 'vl_me

In [19]:
sql("SELECT cvm.drop_view_any('cvm', 'cotas');")
sql("""CREATE MATERIALIZED VIEW cvm.cotas AS
SELECT COALESCE(tp_fundo, tp_fundo_classe) tp_fundo, COALESCE(cnpj_fundo, cnpj_fundo_classe) cnpj_fundo, DATE(dt_comptc) dt_comptc, vl_total, vl_quota, vl_patrim_liq, captc_dia, resg_dia, nr_cotst, id_subclasse
FROM cvm.fi_doc_inf_diario_inf_diario_fi;""")

Operação realizada com sucesso.
Operação realizada com sucesso.


In [9]:
sql("DROP MATERIALIZED VIEW IF EXISTS cvm.carteira CASCADE;")
sql("""CREATE MATERIALIZED VIEW cvm.carteira AS 
WITH cad AS (
	SELECT 
	    cadastro.cnpj_fundo, 
	    depara_gestores.grupo AS gestor, 
	    cadastro.classe, 
	    CASE 
	        WHEN UPPER(denom_social) LIKE '%PREV%' THEN 'Prev' 
	        WHEN cadastro.fundo_exclusivo = 'S' THEN 'Exclusivo' 
	        ELSE 'Outros' 
	    END AS tipo
	FROM cvm.cadastro 
	LEFT JOIN cvm.depara_gestores ON depara_gestores.gestor = cadastro.gestor
	WHERE cadastro.dt_fim IS NULL
)
SELECT cda.dt_comptc, cda.cnpj_fundo, cda.denom_social, cad_inv.gestor cliente, CASE WHEN cliente_segmentado.segmentacao IS NOT NULL THEN cliente_segmentado.segmentacao ELSE CONCAT(cad_inv.gestor, ' ', cad_inv.tipo) END AS cliente_segmentado, cda.cnpj_fundo_cota, cda.nm_fundo_cota, cad_cota.gestor gestor_cota, cda.vl_merc_pos_final, cad_cota.classe peer
FROM cvm.cda_fi_blc_2 cda 
INNER JOIN cad cad_inv ON cad_inv.cnpj_fundo = cda.cnpj_fundo
INNER JOIN cad cad_cota ON cad_cota.cnpj_fundo = cda.cnpj_fundo_cota
LEFT JOIN alocadores.cliente_segmentado ON cliente_segmentado.cnpj_fundo = cad_inv.cnpj_fundo
WHERE cda.tp_fundo IN ('FI', 'FIF', 'CLASSES - FIF', 'CLASSES - FIP')""")

Operação realizada com sucesso.
Operação realizada com sucesso.


In [8]:
# Mapeamento exato das colunas baseado no seu log
cda_configs = {
    "cda_fie": {
        "table": "fi_doc_cda_fie",
        "cols": "tp_fundo_classe AS tp_fundo, cnpj_fundo_classe AS cnpj_fundo, denom_social, (dt_comptc::text)::date dt_comptc, vl_patrim_liq, tp_aplic, tp_ativo, emissor_ligado, tp_negoc, qt_venda_negoc, vl_venda_negoc, qt_aquis_negoc, vl_aquis_negoc, qt_pos_final, vl_merc_pos_final, vl_custo_pos_final, (dt_confid_aplic::text)::date dt_confid_aplic, cd_ativo, ds_ativo, (dt_venc::text)::date dt_venc, pf_pj_emissor, cpf_cnpj_emissor, emissor, risco_emissor, cd_selic, (dt_ini_vigencia::text)::date dt_ini_vigencia, cd_pais, pais, cd_bv_merc, bv_merc"
    },
    "cda_fie_confid": {
        "table": "fi_doc_cda_fie_confid",
        "cols": "tp_fundo_classe AS tp_fundo, cnpj_fundo_classe AS cnpj_fundo, denom_social, (dt_comptc::text)::date dt_comptc, tp_aplic, vl_venda_negoc, vl_aquis_negoc, vl_merc_pos_final, vl_custo_pos_final, (dt_confid_aplic::text)::date dt_confid_aplic"
    },
    "cda_fiim": {
        "table": "fi_doc_cda_fiim",
        "cols": "COALESCE(tp_fundo, tp_fundo_classe) AS tp_fundo, COALESCE(cnpj_fundo, cnpj_fundo_classe) AS cnpj_fundo, denom_social, (dt_comptc::text)::date dt_comptc, vl_patrim_liq, tp_aplic, tp_ativo, emissor_ligado, tp_negoc, qt_venda_negoc, vl_venda_negoc, qt_aquis_negoc, vl_aquis_negoc, qt_pos_final, vl_merc_pos_final, vl_custo_pos_final, (dt_confid_aplic::text)::date dt_confid_aplic, cd_ativo, ds_ativo, (dt_venc::text)::date dt_venc, pf_pj_emissor, cpf_cnpj_emissor, emissor, risco_emissor, cd_selic, (dt_ini_vigencia::text)::date dt_ini_vigencia, cd_pais, pais, cd_bv_merc, bv_merc"
    },
    "cda_fiim_confid": {
        "table": "fi_doc_cda_fiim_confid",
        # Alterado para lidar com o tipo 'double precision' (float)
        "cols": "tp_fundo_classe AS tp_fundo, cnpj_fundo_classe AS cnpj_fundo, denom_social, (dt_comptc::text)::date dt_comptc, tp_aplic, vl_venda_negoc, vl_aquis_negoc, vl_merc_pos_final, vl_custo_pos_final, (dt_confid_aplic::text)::date dt_confid_aplic"
    },
    "cda_fi_blc_1": {
        "table": "fi_doc_cda_fi_blc_1",
        "cols": "COALESCE(tp_fundo, tp_fundo_classe) AS tp_fundo, COALESCE(cnpj_fundo, cnpj_fundo_classe) AS cnpj_fundo, denom_social, (dt_comptc::text)::date dt_comptc, tp_aplic, tp_ativo, emissor_ligado, tp_negoc, qt_venda_negoc, vl_venda_negoc, qt_aquis_negoc, vl_aquis_negoc, qt_pos_final, vl_merc_pos_final, vl_custo_pos_final, (dt_confid_aplic::text)::date dt_confid_aplic, tp_titpub, cd_isin, cd_selic, (dt_emissao::text)::date dt_emissao, (dt_venc::text)::date dt_venc"
    },
    "cda_fi_blc_2": {
        "table": "fi_doc_cda_fi_blc_2",
        "cols": "COALESCE(tp_fundo, tp_fundo_classe) tp_fundo, COALESCE(cnpj_fundo, cnpj_fundo_classe) cnpj_fundo, denom_social, (dt_comptc::text)::date dt_comptc, tp_aplic, tp_ativo, emissor_ligado, tp_negoc, qt_venda_negoc, vl_venda_negoc, qt_aquis_negoc, vl_aquis_negoc, qt_pos_final, vl_merc_pos_final, vl_custo_pos_final, (dt_confid_aplic::text)::date dt_confid_aplic, COALESCE(cnpj_fundo_cota, cnpj_fundo_classe_cota) cnpj_fundo_cota, COALESCE(nm_fundo_cota, nm_fundo_classe_subclasse_cota) nm_fundo_cota, id_subclasse"
    },
    "cda_fi_blc_3": {
        "table": "fi_doc_cda_fi_blc_3",
        "cols": "COALESCE(tp_fundo, tp_fundo_classe) AS tp_fundo, COALESCE(cnpj_fundo, cnpj_fundo_classe) AS cnpj_fundo, denom_social, (dt_comptc::text)::date dt_comptc, tp_aplic, tp_ativo, emissor_ligado, tp_negoc, qt_venda_negoc, vl_venda_negoc, qt_aquis_negoc, vl_aquis_negoc, qt_pos_final, vl_merc_pos_final, vl_custo_pos_final, (dt_confid_aplic::text)::date dt_confid_aplic, cd_swap, ds_swap"
    },
    "cda_fi_blc_4": {
        "table": "fi_doc_cda_fi_blc_4",
        "cols": "COALESCE(tp_fundo, tp_fundo_classe) AS tp_fundo, COALESCE(cnpj_fundo, cnpj_fundo_classe) AS cnpj_fundo, denom_social, (dt_comptc::text)::date dt_comptc, tp_aplic, tp_ativo, emissor_ligado, tp_negoc, qt_venda_negoc, vl_venda_negoc, qt_aquis_negoc, vl_aquis_negoc, qt_pos_final, vl_merc_pos_final, vl_custo_pos_final, (dt_confid_aplic::text)::date dt_confid_aplic, cd_ativo, ds_ativo, cd_isin, (dt_ini_vigencia::text)::date dt_ini_vigencia, (dt_fim_vigencia::text)::date dt_fim_vigencia"
    },
    "cda_fi_blc_5": {
        "table": "fi_doc_cda_fi_blc_5",
        "cols": "COALESCE(tp_fundo, tp_fundo_classe) AS tp_fundo, COALESCE(cnpj_fundo, cnpj_fundo_classe) AS cnpj_fundo, denom_social, (dt_comptc::text)::date dt_comptc, tp_aplic, tp_ativo, emissor_ligado, tp_negoc, qt_venda_negoc, vl_venda_negoc, qt_aquis_negoc, vl_aquis_negoc, qt_pos_final, vl_merc_pos_final, vl_custo_pos_final, (dt_confid_aplic::text)::date dt_confid_aplic, cnpj_emissor, emissor, (dt_venc::text)::date dt_venc, titulo_posfx, cd_indexador_posfx, ds_indexador_posfx, pr_indexador_posfx, pr_cupom_posfx, pr_taxa_prefx, risco_emissor, ag_risco, (dt_risco::text)::date dt_risco, grau_risco"
    },
    "cda_fi_blc_6": {
        "table": "fi_doc_cda_fi_blc_6",
        "cols": "COALESCE(tp_fundo, tp_fundo_classe) AS tp_fundo, COALESCE(cnpj_fundo, cnpj_fundo_classe) AS cnpj_fundo, denom_social, (dt_comptc::text)::date dt_comptc, tp_aplic, tp_ativo, emissor_ligado, tp_negoc, qt_venda_negoc, vl_venda_negoc, qt_aquis_negoc, vl_aquis_negoc, qt_pos_final, vl_merc_pos_final, vl_custo_pos_final, (dt_confid_aplic::text)::date dt_confid_aplic, pf_pj_emissor, cpf_cnpj_emissor, emissor, (dt_venc::text)::date dt_venc, titulo_posfx, cd_indexador_posfx, ds_indexador_posfx, pr_indexador_posfx, pr_cupom_posfx, pr_taxa_prefx, titulo_cetip, titulo_garantia, cnpj_instituicao_financ_coobr"
    },
    "cda_fi_blc_7": {
        "table": "fi_doc_cda_fi_blc_7",
        "cols": "COALESCE(tp_fundo, tp_fundo_classe) AS tp_fundo, COALESCE(cnpj_fundo, cnpj_fundo_classe) AS cnpj_fundo, denom_social, (dt_comptc::text)::date dt_comptc, tp_aplic, tp_ativo, emissor_ligado, tp_negoc, qt_venda_negoc, vl_venda_negoc, qt_aquis_negoc, vl_aquis_negoc, qt_pos_final, vl_merc_pos_final, vl_custo_pos_final, (dt_confid_aplic::text)::date dt_confid_aplic, invest_coletivo, invest_coletivo_gestor, emissor, (dt_venc::text)::date dt_venc, cd_pais, pais, cd_bv_merc, bv_merc, cd_ativo_bv_merc, risco_emissor, ag_risco, (dt_risco::text)::date dt_risco, grau_risco, ds_ativo_exterior, qt_ativo_exterior, vl_ativo_exterior"
    },
    "cda_fi_blc_8": {
        "table": "fi_doc_cda_fi_blc_8",
        "cols": "COALESCE(tp_fundo, tp_fundo_classe) AS tp_fundo, COALESCE(cnpj_fundo, cnpj_fundo_classe) AS cnpj_fundo, denom_social, (dt_comptc::text)::date dt_comptc, tp_aplic, tp_ativo, emissor_ligado, tp_negoc, qt_venda_negoc, vl_venda_negoc, qt_aquis_negoc, vl_aquis_negoc, qt_pos_final, vl_merc_pos_final, vl_custo_pos_final, (dt_confid_aplic::text)::date dt_confid_aplic, ds_ativo, pf_pj_emissor, cpf_cnpj_emissor, emissor"
    },
    "cda_fi_confid": {
        "table": "fi_doc_cda_fi_confid",
        "cols": "tp_fundo_classe AS tp_fundo, cnpj_fundo_classe AS cnpj_fundo, denom_social, (dt_comptc::text)::date dt_comptc, tp_aplic, vl_venda_negoc, vl_aquis_negoc, vl_merc_pos_final, vl_custo_pos_final, (dt_confid_aplic::text)::date dt_confid_aplic"
    },
    "cda_fi_pl": {
        "table": "fi_doc_cda_fi_pl",
        "cols": "COALESCE(tp_fundo, tp_fundo_classe) AS tp_fundo, COALESCE(cnpj_fundo, cnpj_fundo_classe) AS cnpj_fundo, denom_social, (dt_comptc::text)::date dt_comptc, vl_patrim_liq"
    }
}

for view_name, info in cda_configs.items():
    sql(f"DROP VIEW IF EXISTS cvm.{view_name} CASCADE;")
    
    query = f"""
    CREATE VIEW cvm.{view_name} AS
    SELECT {info['cols']}
    FROM cvm.{info['table']};
    """
    sql(query)

Operação realizada com sucesso.
Operação realizada com sucesso.
Operação realizada com sucesso.
Operação realizada com sucesso.
Operação realizada com sucesso.
Operação realizada com sucesso.
Operação realizada com sucesso.
Operação realizada com sucesso.
Operação realizada com sucesso.
Operação realizada com sucesso.
Operação realizada com sucesso.
Operação realizada com sucesso.
Operação realizada com sucesso.
Operação realizada com sucesso.
Operação realizada com sucesso.
Operação realizada com sucesso.
Operação realizada com sucesso.
Operação realizada com sucesso.
Operação realizada com sucesso.
Operação realizada com sucesso.
Operação realizada com sucesso.
Operação realizada com sucesso.
Operação realizada com sucesso.
Operação realizada com sucesso.
Operação realizada com sucesso.
Operação realizada com sucesso.
Operação realizada com sucesso.
Operação realizada com sucesso.


In [8]:
sql("""
DROP VIEW IF EXISTS cvm.espelhos CASCADE;
DROP MATERIALIZED VIEW IF EXISTS cvm.espelhos CASCADE;

CREATE MATERIALIZED VIEW cvm.espelhos AS
WITH 
cotas AS (
    -- Substituição do GROUP BY MONTH/YEAR por DATE_TRUNC
    SELECT 
        cnpj_fundo, 
        MAX(dt_comptc::text::date) AS dt_comptc, -- Pega a maior data do mês (ex: 30/01)
        MAX(vl_patrim_liq) AS vl_patrim_liq   -- Pega o PL desse dia
    FROM cvm.cda_fi_pl -- Ajuste se sua tabela de origem for outra
    WHERE dt_comptc::text::date >= CURRENT_DATE - INTERVAL '1 year'
    GROUP BY 
        cnpj_fundo, 
        DATE_TRUNC('month', dt_comptc::text::date) -- Agrupa por mês/ano
),
carteiras_fechadas AS (
    SELECT 
        cnpj_fundo, 
        MIN(dt_comptc::text::date) AS dt_comptc 
    FROM cvm.cda_fi_confid 
    WHERE dt_confid_aplic::text::date > CURRENT_DATE 
    GROUP BY cnpj_fundo
),
carteiras_abertas AS (
    SELECT cda.* FROM cvm.cda_fi_blc_2 cda
    INNER JOIN carteiras_fechadas cf 
        ON cda.cnpj_fundo = cf.cnpj_fundo 
        AND cda.dt_comptc::text::date < cf.dt_comptc
    UNION 
    SELECT * FROM cvm.cda_fi_blc_2 
    WHERE cnpj_fundo NOT IN (SELECT cnpj_fundo FROM carteiras_fechadas)
),
ultima_aberta AS (
    SELECT ca.* FROM carteiras_abertas ca
    INNER JOIN (
        SELECT cnpj_fundo, MAX(dt_comptc::text::date) AS dt_comptc 
        FROM carteiras_abertas 
        GROUP BY cnpj_fundo
    ) max_carteira
    ON max_carteira.dt_comptc = ca.dt_comptc::text::date 
    AND max_carteira.cnpj_fundo = ca.cnpj_fundo
),
pct_pl AS (
    SELECT 
        ua.cnpj_fundo, 
        ua.cnpj_fundo_cota, 
        -- Simulação do try_divide com NULLIF para evitar divisão por zero
        (ua.vl_merc_pos_final / NULLIF(c.vl_patrim_liq, 0)) AS pct, 
        ua.dt_comptc 
    FROM ultima_aberta ua
    INNER JOIN cotas c ON c.cnpj_fundo = ua.cnpj_fundo 
    -- Join comparando Mês e Ano usando DATE_TRUNC
    AND DATE_TRUNC('month', c.dt_comptc) = DATE_TRUNC('month', ua.dt_comptc::text::date)
),
espelhos AS ( 
    SELECT cnpj_fundo, cnpj_fundo_cota 
    FROM pct_pl 
    WHERE pct > 0.9
),
consolidada AS (
    SELECT 
        e1.cnpj_fundo, 
        COALESCE(e4.cnpj_fundo_cota, e3.cnpj_fundo_cota, e2.cnpj_fundo_cota, e1.cnpj_fundo_cota) AS cnpj_fundo_cota
    FROM espelhos e1
    LEFT JOIN espelhos e2 ON e2.cnpj_fundo = e1.cnpj_fundo_cota
    LEFT JOIN espelhos e3 ON e3.cnpj_fundo = e2.cnpj_fundo_cota
    LEFT JOIN espelhos e4 ON e4.cnpj_fundo = e3.cnpj_fundo_cota
)
SELECT * FROM consolidada;""")

Operação realizada com sucesso.


In [3]:
sql("SELECT cvm.drop_view_any('cvm', 'ativos_carteira')")
sql ("""
CREATE MATERIALIZED VIEW cvm.ativos_carteira AS
    SELECT 'blc_1' bloco, tp_fundo, cnpj_fundo, dt_comptc, tp_aplic, tp_ativo, qt_pos_final, vl_merc_pos_final, cd_isin cd_ativo, 'ISIN' tp_cd_ativo, tp_titpub nm_ativo FROM cvm.cda_fi_blc_1
UNION SELECT 'blc_2' bloco, tp_fundo, cnpj_fundo, dt_comptc, tp_aplic, tp_ativo, qt_pos_final, vl_merc_pos_final, cnpj_fundo_cota cd_ativo, 'CNPJ' tp_cd_ativo, nm_fundo_cota nm_ativo FROM cvm.cda_fi_blc_2
UNION SELECT 'blc_3' bloco, tp_fundo, cnpj_fundo, dt_comptc, tp_aplic, tp_ativo, qt_pos_final, vl_merc_pos_final, cd_swap cd_ativo, 'SWAP' tp_cd_ativo, ds_swap nm_ativo FROM cvm.cda_fi_blc_3	-- 
UNION SELECT 'blc_4' bloco, tp_fundo, cnpj_fundo, dt_comptc, tp_aplic, tp_ativo, qt_pos_final, vl_merc_pos_final, cd_ativo, 'TICKER' tp_cd_ativo, ds_ativo nm_ativo FROM cvm.cda_fi_blc_4
UNION SELECT 'blc_5' bloco, tp_fundo, cnpj_fundo, dt_comptc, tp_aplic, tp_ativo, qt_pos_final, vl_merc_pos_final, cnpj_emissor cd_ativo, 'CNPJ' tp_cd_ativo, CONCAT(tp_ativo, ' - ', emissor, ' - ') nm_ativo FROM cvm.cda_fi_blc_5
UNION SELECT 'blc_6' bloco, tp_fundo, cnpj_fundo, dt_comptc, tp_aplic, tp_ativo, qt_pos_final, vl_merc_pos_final, cpf_cnpj_emissor cd_ativo, 'CNPJ' tp_cd_ativo, CONCAT(tp_ativo, ' - ', emissor, ' - ', cd_indexador_posfx, ' - ', ds_indexador_posfx) nm_ativo FROM cvm.cda_fi_blc_6
UNION SELECT 'blc_7' bloco, tp_fundo, cnpj_fundo, dt_comptc, tp_aplic, tp_ativo, qt_pos_final, vl_merc_pos_final, CONCAT(cd_pais, ' - ', cd_bv_merc, ' - ', cd_ativo_bv_merc) cd_ativo, 'PAIS - BOLSA - CODIGO' tp_cd_ativo, emissor nm_ativo FROM cvm.cda_fi_blc_7
UNION SELECT 'blc_8' bloco, tp_fundo, cnpj_fundo, dt_comptc, tp_aplic, tp_ativo, qt_pos_final, vl_merc_pos_final, cpf_cnpj_emissor cd_ativo, CASE WHEN cpf_cnpj_emissor IS NULL THEN NULL ELSE 'CNPJ' END AS tp_cd_ativo, CASE WHEN cpf_cnpj_emissor IS NOT NULL THEN CONCAT(ds_ativo, ' - ', emissor) ELSE ds_ativo END AS nm_ativo FROM cvm.cda_fi_blc_8
""")

Operação realizada com sucesso.
Operação realizada com sucesso.
